In [42]:
import re
import nltk
import pandas as pd
nltk.download('gutenberg')

corrupted_file = open('ausen-sense-corrupted.txt').read()
correct_file = nltk.corpus.gutenberg.raw('austen-sense.txt')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/erictay1997/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
def tokenizer(textFile):
    words = re.findall(r"[\w']+|[\n-.\",!?:;]", textFile)
    tokens = []
    for word in words:
        foundContraction = re.search(r"(\w+)('ll|'LL|'re|'RE|'ve|'VE|n't|N'T|'s|'S|'d|'D|'m|'M|'a|')", word)
        if foundContraction:
            tokens.append(foundContraction.group(1))
            tokens.append(foundContraction.group(2))
        else:
            tokens.append(word)
    
    print("finished tokenizing")
    print(len(tokens))
    return tokens

In [3]:
corrupted_tokens = tokenizer(corrupted_file)
correct_tokens = tokenizer(correct_file)

finished tokenizing
267642
finished tokenizing
267704


In [4]:
for i in range(62):
    corrupted_tokens.append('0')

In [5]:
# These spaces are to handle misalignment issues
# This occurs when the corrupted file deletes words, which we cannot correct
counter = 0
for i in range(len(correct_tokens)):
    if corrupted_tokens[i] != correct_tokens[i]:
        if correct_tokens[i] == " ":
            counter += 1
            corrupted_tokens.insert(i, " ")
            corrupted_tokens.pop()
print(counter)

61


In [6]:
# There are 18929 tokens that are different
df = pd.DataFrame({'corrupted' : corrupted_tokens, 'correct' : correct_tokens }, columns=['corrupted','correct'])
misaligned = df[df['corrupted'] != df['correct']]
len(misaligned)

18929

In [7]:
# Print for Sanity Check
# for i in range(len(misaligned)):
#     print(misaligned.iloc[i,])

In [8]:
all_words = open('dictionary.txt').read().split('\n')
# dictionary.txt is a txt file of all valid words taken from the link on Piazza
# https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt

In [9]:
import pandas as pd
df = pd.read_csv('unigram_freq.csv', index_col = False)
# unigram_freq.csv is a csv file from Kaggle, containing english words and word frequency
# Data is derived from the Google Web Trillion Word Corpus.
# https://www.kaggle.com/rtatman/english-word-frequency

In [10]:
df = df[df['word'].isin(all_words)] # Clean kaggle dataset

In [11]:
word_counter = {}
for i in range(len(df)):
    word_counter[df.iloc[i,0]] = df.iloc[i,1]

In [12]:
for word in all_words:
    if word not in word_counter:
        word_counter[word] = 1

In [13]:
def spell_corrector(word_list):
    return [correct(word) for word in word_list]

In [14]:
# Corrects word if it's alphanumeric
# Else, do nothing
# Capitalizes it accordingly
def correct(word):
    if not word.isalnum():
        return word
    if word.lower() in word_counter:
        return word
    corrected_word = best_candidate(word.lower())
    if word.isupper():
        return corrected_word.upper()
    if word[0].isupper():
        return corrected_word[0].upper() + corrected_word[1:]
    return corrected_word

In [15]:
# Returns best candidate for a given word
# Prioritizes lower Levenshtein distance, and then word frequency
# If there are no words with Levenshtein distance ≤ 2 in word_counter, return the word itself
def best_candidate(word):
    return (best_candidate_from_list(distance1(word)) or best_candidate_from_list(distance2(word)) or word)

In [16]:
# Returns best candidate from a list of words
# Weighted by word frequency
def best_candidate_from_list(words):
    count = -1
    candidate = None
    for w in words:
        if w in word_counter and word_counter[w] > count:
            count = word_counter[w]
            candidate = w
    return candidate

In [17]:
# Set of words of one Levenshtein distance from word
def distance1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    ret = set()
    for i in range(len(word)):
        ret.add(word[:i] + word[i+1:]) #Deletion
        for letter in letters:
            ret.add(word[:i] + letter + word[i:]) #Insertion
            ret.add(word[:i] + letter + word[i+1:]) #Substitution
    return ret

In [18]:
# Set of words of two Levenshtein distance from word
def distance2(word): 
    ret = set()
    for distance1_word in distance1(word):
        ret.update(distance1(distance1_word))
    return ret

In [20]:
# These are the tokens are are different
corrupted_tokens_inaccurate = []
correct_tokens_shortlist = []

In [21]:
for i in range(len(corrupted_tokens)):
    if corrupted_tokens[i] != correct_tokens[i]:
        corrupted_tokens_inaccurate.append(corrupted_tokens[i])
        correct_tokens_shortlist.append(correct_tokens[i])

In [22]:
corrected_tokens = spell_corrector(corrupted_tokens_inaccurate)

In [47]:
counter = 0
for i in range(len(correct_tokens_shortlist)):
    if correct_tokens_shortlist[i] == corrected_tokens[i]:
        counter += 1

In [48]:
# We correct 63% of tokens
counter/len(correct_tokens_shortlist)

0.6260235617306777

In [27]:
# 7079 tokens are still different
df = pd.DataFrame({'corrupted' : corrected_tokens, 'correct' : correct_tokens_shortlist }, columns=['corrupted','correct'])
misaligned = df[df['corrupted'] != df['correct']]
len(misaligned)

7079

In [28]:
for i in range(len(misaligned)):
    print(misaligned.iloc[i,])

corrupted     i
correct      in
Name: 1, dtype: object
corrupted      stat
correct      estate
Name: 5, dtype: object
corrupted     nd
correct      and
Name: 7, dtype: object
corrupted     te
correct      the
Name: 15, dtype: object
corrupted     o
correct      of
Name: 18, dtype: object
corrupted     das
correct      days
Name: 22, dtype: object
corrupted     Diswood
correct      Dashwood
Name: 25, dtype: object
corrupted     for
correct      from
Name: 26, dtype: object
corrupted     hi
correct      him
Name: 28, dtype: object
corrupted     n
correct      to
Name: 31, dtype: object
corrupted    has
correct      his
Name: 33, dtype: object
corrupted     te
correct      the
Name: 35, dtype: object
corrupted     ms
correct      his
Name: 37, dtype: object
corrupted     herefore
correct      therefore
Name: 40, dtype: object
corrupted    forsung
correct      fortune
Name: 43, dtype: object
corrupted    right
correct      might
Name: 44, dtype: object
corrupted     bt
correct      but
Nam

Name: 489, dtype: object
corrupted     taa
correct      than
Name: 494, dtype: object
corrupted    hoey
correct      They
Name: 496, dtype: object
corrupted      
correct      no
Name: 499, dtype: object
corrupted    haze
correct      have
Name: 500, dtype: object
corrupted     an
correct      any
Name: 502, dtype: object
corrupted     the
correct      they
Name: 503, dtype: object
corrupted     hem
correct      them
Name: 506, dtype: object
corrupted     mor
correct      more
Name: 507, dtype: object
corrupted     gie
correct      give
Name: 510, dtype: object
corrupted    ara
correct      are
Name: 512, dtype: object
corrupted     noting
correct      nothing
Name: 514, dtype: object
corrupted    mn
correct      me
Name: 516, dtype: object
corrupted     really
correct      readily
Name: 521, dtype: object
corrupted     even
correct      given
Name: 522, dtype: object
corrupted     e
correct      be
Name: 523, dtype: object
corrupted     Ms
correct      Mrs
Name: 524, dtype: object
cor

Name: 838, dtype: object
corrupted     hm
correct      him
Name: 839, dtype: object
corrupted     worry
correct      worthy
Name: 840, dtype: object
corrupted     ure
correct      sure
Name: 841, dtype: object
corrupted     minor
correct      Elinor
Name: 846, dtype: object
corrupted    worts
correct      worth
Name: 852, dtype: object
corrupted     del
correct      deal
Name: 856, dtype: object
corrupted    been
correct      seen
Name: 860, dtype: object
corrupted    YI
correct       I
Name: 862, dtype: object
corrupted     Is
correct      His
Name: 865, dtype: object
corrupted     no
correct      not
Name: 866, dtype: object
corrupted     o
correct      of
Name: 868, dtype: object
corrupted    this
correct       his
Name: 869, dtype: object
corrupted    hia
correct      him
Name: 872, dtype: object
corrupted     ad
correct      and
Name: 882, dtype: object
corrupted     women
correct      moment
Name: 884, dtype: object
corrupted    ho
correct      to
Name: 885, dtype: object
corrupt

Name: 1393, dtype: object
corrupted     seems
correct      seemed
Name: 1396, dtype: object
corrupted    ge
correct      be
Name: 1399, dtype: object
corrupted     o
correct      so
Name: 1402, dtype: object
corrupted     thy
correct      they
Name: 1404, dtype: object
corrupted     preset
correct      present
Name: 1413, dtype: object
corrupted    penguin
correct      sending
Name: 1416, dtype: object
corrupted     iddleton
correct      Middleton
Name: 1417, dtype: object
corrupted    re
correct      be
Name: 1419, dtype: object
corrupted    ob
correct      of
Name: 1427, dtype: object
corrupted     hr
correct      her
Name: 1431, dtype: object
corrupted      went
correct      twenty
Name: 1435, dtype: object
corrupted    hall
correct      tall
Name: 1436, dtype: object
corrupted     f
correct      of
Name: 1438, dtype: object
corrupted     y
correct      by
Name: 1441, dtype: object
corrupted      othin
correct      nothing
Name: 1444, dtype: object
corrupted    verby
correct       v

corrupted     other
correct      mother
Name: 1971, dtype: object
corrupted     i
correct      in
Name: 1972, dtype: object
corrupted     hile
correct      while
Name: 1973, dtype: object
corrupted     a
correct      an
Name: 1974, dtype: object
corrupted      event
correct      evident
Name: 1975, dtype: object
corrupted     chic
correct      which
Name: 1976, dtype: object
corrupted    with
correct       its
Name: 1981, dtype: object
corrupted    grateful
correct      graceful
Name: 1982, dtype: object
corrupted    ani
correct      and
Name: 1985, dtype: object
corrupted     f
correct      of
Name: 1987, dtype: object
corrupted     Dashpot
correct      Dashwood
Name: 1988, dtype: object
corrupted     ct
correct      act
Name: 1991, dtype: object
corrupted     Be
correct      She
Name: 1994, dtype: object
corrupted     he
correct      her
Name: 1996, dtype: object
corrupted    Dashwoud
correct      Dashwood
Name: 1998, dtype: object
corrupted     wulk
correct      would
Name: 2001, dt

...

Name: 16423, dtype: object
corrupted     tess
correct      these
Name: 16424, dtype: object
corrupted     red
correct      died
Name: 16428, dtype: object
corrupted     te
correct      the
Name: 16431, dtype: object
corrupted    bo
correct      to
Name: 16432, dtype: object
corrupted     ao
correct      ago
Name: 16434, dtype: object
corrupted    Willounghby
correct       Willoughby
Name: 16436, dtype: object
corrupted     exceed
correct      excited
Name: 16438, dtype: object
corrupted    make
correct      made
Name: 16439, dtype: object
corrupted    aes
correct       as
Name: 16442, dtype: object
corrupted     er
correct      her
Name: 16444, dtype: object
corrupted     hi
correct      his
Name: 16447, dtype: object
corrupted     he
correct      She
Name: 16449, dtype: object
corrupted    iq
correct      it
Name: 16455, dtype: object
corrupted     low
correct      love
Name: 16457, dtype: object
corrupted     t
correct      it
Name: 16458, dtype: object
corrupted     log
correct     

Name: 16989, dtype: object
corrupted      the
correct      there
Name: 16996, dtype: object
corrupted     posting
correct      pointing
Name: 16997, dtype: object
corrupted    protecting
correct      projecting
Name: 16999, dtype: object
corrupted     fist
correct      first
Name: 17002, dtype: object
corrupted     pay
correct      pain
Name: 17004, dtype: object
corrupted    nou
correct      now
Name: 17008, dtype: object
corrupted    ho
correct      to
Name: 17011, dtype: object
corrupted     A
correct      As
Name: 17012, dtype: object
corrupted     hav
correct      have
Name: 17013, dtype: object
corrupted     fr
correct      far
Name: 17014, dtype: object
corrupted     yo
correct      you
Name: 17016, dtype: object
corrupted     y
correct      my
Name: 17017, dtype: object
corrupted     e
correct      me
Name: 17020, dtype: object
corrupted     issued
correct      assured
Name: 17021, dtype: object
corrupted     e
correct      be
Name: 17025, dtype: object
corrupted    double
corr

Name: 17641, dtype: object
corrupted     sel
correct      self
Name: 17644, dtype: object
corrupted    hins
correct       his
Name: 17655, dtype: object
corrupted     ore
correct      more
Name: 17658, dtype: object
corrupted     luring
correct      lurking
Name: 17662, dtype: object
corrupted     hat
correct      That
Name: 17664, dtype: object
corrupted     marred
correct      married
Name: 17670, dtype: object
corrupted     Edgar
correct      Edward
Name: 17672, dtype: object
corrupted     moter
correct      mother
Name: 17674, dtype: object
corrupted     pace
correct      place
Name: 17676, dtype: object
corrupted     et
correct      yet
Name: 17679, dtype: object
corrupted     i
correct      in
Name: 17681, dtype: object
corrupted     biting
correct      uniting
Name: 17683, dtype: object
corrupted     o
correct      of
Name: 17691, dtype: object
corrupted    Kennings
correct      Jennings
Name: 17692, dtype: object
corrupted    ani
correct      and
Name: 17693, dtype: object
corr

Name: 18274, dtype: object
corrupted     honor
correct      honour
Name: 18281, dtype: object
corrupted       amid
correct      admired
Name: 18282, dtype: object
corrupted     yoursel
correct      yourself
Name: 18285, dtype: object
corrupted    HI
correct       I
Name: 18286, dtype: object
corrupted     pleaser
correct      pleasure
Name: 18294, dtype: object
corrupted    mat
correct       at
Name: 18295, dtype: object
corrupted     e
correct      be
Name: 18297, dtype: object
corrupted     mut
correct      must
Name: 18305, dtype: object
corrupted     elt
correct      felt
Name: 18307, dtype: object
corrupted    the
correct       he
Name: 18310, dtype: object
corrupted     any
correct      land
Name: 18313, dtype: object
corrupted    tithed
correct      tithes
Name: 18314, dtype: object
corrupted     herd
correct      heard
Name: 18317, dtype: object
corrupted     hey
correct      They
Name: 18321, dtype: object
corrupted     intimae
correct      intimate
Name: 18323, dtype: object


corrupted     event
correct      extent
Name: 18785, dtype: object
corrupted     eery
correct      every
Name: 18791, dtype: object
corrupted    tis
correct      his
Name: 18792, dtype: object
corrupted     o
correct      of
Name: 18794, dtype: object
corrupted       upped
correct      supposed
Name: 18797, dtype: object
corrupted     minor
correct      Elinor
Name: 18800, dtype: object
corrupted      tells
correct      useless
Name: 18804, dtype: object
corrupted     the
correct      than
Name: 18805, dtype: object
corrupted     had
correct      half
Name: 18806, dtype: object
corrupted    colonel
correct      Colonel
Name: 18810, dtype: object
corrupted    other
correct        her
Name: 18814, dtype: object
corrupted      thing
correct      nothing
Name: 18815, dtype: object
corrupted    co
correct      to
Name: 18816, dtype: object
corrupted     sealed
correct      settled
Name: 18819, dtype: object
corrupted     obligation
correct      obligations
Name: 18823, dtype: object
corrupt

In [38]:
corrected_tokens_all = spell_corrector(corrupted_tokens)

In [39]:
# 7833 are different, so our spell corrector doesn't change too many 'correct' words
df = pd.DataFrame({'corrupted' : corrected_tokens_all, 'correct' : correct_tokens }, columns=['corrupted','correct'])
misaligned = df[df['corrupted'] != df['correct']]
len(misaligned)

7833

In [49]:
# There are 267704 tokens altogether, of which 7833 are different
len(corrected_tokens_all)

267704

In [43]:
df = pd.DataFrame(corrected_tokens_all, columns=["corrected_tokens"])
df.to_csv('corrected_tokens.csv', index=False)

In [ ]:
# Time to detokenize and see output!